In [84]:
import csv
from collections import defaultdict

toJson = open('anime.json', 'w')
count = 0
toJson.write('[')
with open('anime.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for row in csvReader:
        if row[0] == 'anime_id':
            continue
        
        
        genre_list = row[2].split(',')
        genres = '['
        for g in range(len(genre_list)):
            genre = genre_list[g].replace(" ", "")
            genres += '"'
            genres += genre
            if g != len(genre_list)-1:
                genres += '",'
            else:
                genres += '"]'
                
        rating = row[5]
        if rating == '':
            rating = -1
        
        if (count != 0):
            toJson.write(',\n')
        
        toJson.write('{ "anime_id" : "' + str(row[0]) + '", "name" : "' + str(row[1]) + '", "genre" : '
              + str(genres) + ', "type" : "' + str(row[3]) + '", "episodes" : "'
              + str(row[4]) + '", "rating" : ' + str(rating) + ', "members" : ' + str(row[6]) + '}')
        
        count += 1
    
toJson.write(']')
toJson.close()

In [1]:
import json

rating = json.load(open('rating_200k.json'))
animedata = json.load(open('anime.json'))


In [2]:
from collections import defaultdict

watched = defaultdict(set)
rating_by_user = defaultdict(list)    # list of animes with rating that each user watched
rating_by_anime = defaultdict(list)    # list of dictionary with anime as keys, and containing user and ratings


for entry in rating:
    watched[entry['user_id']].add(entry['anime_id'])
    
    anime_dict = {}
    if (entry['rating'] != '-1'):
        anime_dict['anime_id'] = entry['anime_id']
        anime_dict['rating'] = entry['rating']
        rating_by_user[entry['user_id']].append(anime_dict)
    
    user_dict = {}
    if (entry['rating'] != '-1'):
        user_dict['rating'] = entry['rating']
        user_dict['user_id'] = entry['user_id'] 
        rating_by_anime[entry['anime_id']].append(user_dict)
    
    
    
    
    
   

In [3]:
genres = set()    # set of all genres
genreCount = defaultdict(int)     # count of each genre

for entry in animedata:
    for genre in entry['genre']:
        genres.add(genre)
        genreCount[genre] += 1

In [4]:
anime_view_dict = defaultdict(list)     # list of users that watched certain anime

for entry in rating:
    anime_id = entry['anime_id']
    user_id = entry['user_id']
    anime_view_dict[anime_id].append(user_id)

In [5]:
genre_by_anime = defaultdict(list)      # list of animes corresponding to certain genre

for entry in animedata: 
    for genre in entry['genre']:
        genre_by_anime[genre].append(entry['anime_id'])

In [9]:
average_rating_by_user = defaultdict(float)

for user in rating_by_user:
    
    animes = rating_by_user[user]
    total = 0
    count = 0
    
    for anime in animes:    
        total += float(anime['rating'])
        count += 1
            
    if count != 0:
        average_rating_by_user[user] = total / count
        
        

In [10]:
beta_u = defaultdict(float)

total = 0
count = 0
for user in average_rating_by_user:
    total += average_rating_by_user[user]
    count += 1
    
total_average_rating_byuser = total / count

for user in average_rating_by_user:
    beta_u[user] = average_rating_by_user[user] - total_average_rating_byuser

In [11]:
anime_average_rating = defaultdict(float)      # average rating of each anime

for entry in animedata:
    if (entry['rating'] != -1 and entry['rating'] != 0):
        anime_average_rating[entry['anime_id']] = entry['rating']

In [12]:
beta_i = defaultdict(float)

total = 0
count = 0
for anime in anime_average_rating:
    total += anime_average_rating[anime]
    count += 1
    
total_average_rating_byanime = total / count

for anime in anime_average_rating:
    beta_i[anime] = anime_average_rating[anime] - total_average_rating_byanime

In [13]:
alpha = anime_average_rating

In [40]:
# Make a train / validation set

rating_filtered = []
for entry in rating:
    if (entry['rating'] != '-1'):
        rating_filtered.append(entry)

In [16]:
# Approach 1

# rating_filtered

In [19]:
# Approach 2

import random

def scrambled(orig):
    dest = orig[:]
    random.shuffle(dest)
    return dest

rating_scrambled = scrambled(rating_filtered)

In [145]:
# Approach 3

rating_toModify = rating_filtered[:]

current_user_id = 1
rating_3 = []

checknumber = 150000

while(len(rating_toModify) > 3000):
    
    for j in range(0,len(rating_toModify)-1):

        if j >= len(rating_toModify)-1:
            break
        
        this_rating = rating_toModify[j]
        current_user_id_str = str(current_user_id)
        
        if this_rating['user_id'] == current_user_id_str:

            rating_3.append(this_rating)
            rating_toModify.pop(j)
            current_user_id += 1
            
        if int(this_rating['user_id']) > current_user_id:
            current_user_id += 1
            
        

    if current_user_id >= 2012:
        current_user_id = 1
    
remaining = rating_toModify[:]
rating_3 = rating_3 + remaining

In [156]:
writeFile = open('ratingset_1.json', 'w')
for entry in rating_filtered:
    writeFile.write(str(entry))
    
writeFile.close()

writeFile = open('ratingset_2.json', 'w')
for entry in rating_scrambled:
    writeFile.write(str(entry))
    
writeFile.close()

writeFile = open('ratingset_3.json', 'w')
for entry in rating_3:
    writeFile.write(str(entry))
    
writeFile.close()


In [154]:
rating_filtered

[{'anime_id': '8074', 'rating': '10', 'user_id': '1'},
 {'anime_id': '11617', 'rating': '10', 'user_id': '1'},
 {'anime_id': '11757', 'rating': '10', 'user_id': '1'},
 {'anime_id': '15451', 'rating': '10', 'user_id': '1'},
 {'anime_id': '11771', 'rating': '10', 'user_id': '2'},
 {'anime_id': '20', 'rating': '8', 'user_id': '3'},
 {'anime_id': '154', 'rating': '6', 'user_id': '3'},
 {'anime_id': '170', 'rating': '9', 'user_id': '3'},
 {'anime_id': '199', 'rating': '10', 'user_id': '3'},
 {'anime_id': '225', 'rating': '9', 'user_id': '3'},
 {'anime_id': '341', 'rating': '6', 'user_id': '3'},
 {'anime_id': '430', 'rating': '7', 'user_id': '3'},
 {'anime_id': '527', 'rating': '7', 'user_id': '3'},
 {'anime_id': '552', 'rating': '7', 'user_id': '3'},
 {'anime_id': '813', 'rating': '10', 'user_id': '3'},
 {'anime_id': '1119', 'rating': '7', 'user_id': '3'},
 {'anime_id': '1121', 'rating': '7', 'user_id': '3'},
 {'anime_id': '1122', 'rating': '7', 'user_id': '3'},
 {'anime_id': '1132', 'ratin